# Data Preparation & Cleaning

The first thing we did for the project was cleaning and preparation of the dataset to help us gain meaningful insights from the dataset and help us answer the question we posed.

#### Questions: Create a customer segmentation campaign to label customers based on their risk profiles (e.g. riskier customers, less risky customers, normal drivers, etc.).

#### Dataset: [Vehicle Insurance Policy 2020](https://www.kaggle.com/datasets/lakshmanraj/vehicle-insurance-policy?select=Vehicle_policies_2020.csv)

## Table of Contents:

1. Dropping Unused Columns 
2. Fill empty cells with Median
3. Split Dataset in Two
4. Removing Outliers
5. Addition of 4 New Derived Columns: "month", "year", "monthDiff", "claims"
6. Scaling credit_score, traffic_index and veh_age to a range 1-100

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
# import CSV file
data = pd.read_csv('Vehicle_policies_2020.csv')
data

,pol_number,pol_eff_dt,gender,agecat,date_of_birth,credit_score,area,traffic_index,veh_age,veh_body,veh_value,claim_office,numclaims,claimcst0,annual_premium
0,43124327,12/30/2020,F,4.0,7/12/1968,381.0,D,133.6,2,HBACK,1.331,NaN,0,0.0,716.53
1,21919609,12/30/2020,F,2.0,11/5/1982,549.0,D,163.6,1,UTE,3.740,NaN,0,0.0,716.53
2,72577057,12/30/2020,M,2.0,11/26/1983,649.0,B,117.5,4,COUPE,0.880,NaN,0,0.0,716.53
3,92175225,12/30/2020,M,4.0,11/2/1960,743.0,B,100.7,3,SEDAN,1.045,NaN,0,0.0,716.53
4,66223239,12/30/2020,F,4.0,1/4/1968,817.0,C,115.5,4,HBACK,0.473,NaN,0,0.0,716.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60387,73248694,1/2/2020,F,5.0,5/10/1956,809.0,C,145.5,4,HBACK,0.891,NaN,0,0.0,716.53
60388,71411764,1/2/2020,M,4.0,3/22/1961,681.0,D,86.3,1,HBACK,1.881,NaN,0,0.0,716.53
60389,89353155,1/2/2020,M,4.0,9/29/1965,773.0,F,110.0,1,STNWG,5.170,NaN,0,0.0,716.53
60390,40916605,1/2/2020,M,3.0,8/1/1978,714.0,B,NaN,1,HBACK,1.903,NaN,0,0.0,716.53


In [3]:
data.shape

(60392, 15)

In [4]:
data.info

<bound method DataFrame.info of        pol_number  pol_eff_dt gender  agecat date_of_birth  credit_score area  \
0        43124327  12/30/2020      F     4.0     7/12/1968         381.0    D   
1        21919609  12/30/2020      F     2.0     11/5/1982         549.0    D   
2        72577057  12/30/2020      M     2.0    11/26/1983         649.0    B   
3        92175225  12/30/2020      M     4.0     11/2/1960         743.0    B   
4        66223239  12/30/2020      F     4.0      1/4/1968         817.0    C   
...           ...         ...    ...     ...           ...           ...  ...   
60387    73248694    1/2/2020      F     5.0     5/10/1956         809.0    C   
60388    71411764    1/2/2020      M     4.0     3/22/1961         681.0    D   
60389    89353155    1/2/2020      M     4.0     9/29/1965         773.0    F   
60390    40916605    1/2/2020      M     3.0      8/1/1978         714.0    B   
60391    33623054    1/2/2020      F     3.0    12/12/1973           NaN    D

## 1. Dropping Unused Columns 
Firstly,  we noticed that our dataset contains some NaNs. These NaNs mean that the survey respondent chose not to answer that question. Since we are only concerned with respondents who have answered all the questions in the survey, we have simply dropped these NaNs from our dataset.

In [5]:
# drop unused columns
data = data.drop(columns = ["pol_number", "date_of_birth", "claim_office", "annual_premium"])
# reset the index of the rows of the DataFrame
data = data.reset_index(drop=True)


In [6]:
print(f"The shape of the new dataset: {data.shape}")

data

The shape of the new dataset: (60392, 11)


,pol_eff_dt,gender,agecat,credit_score,area,traffic_index,veh_age,veh_body,veh_value,numclaims,claimcst0
0,12/30/2020,F,4.0,381.0,D,133.6,2,HBACK,1.331,0,0.0
1,12/30/2020,F,2.0,549.0,D,163.6,1,UTE,3.740,0,0.0
2,12/30/2020,M,2.0,649.0,B,117.5,4,COUPE,0.880,0,0.0
3,12/30/2020,M,4.0,743.0,B,100.7,3,SEDAN,1.045,0,0.0
4,12/30/2020,F,4.0,817.0,C,115.5,4,HBACK,0.473,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
60387,1/2/2020,F,5.0,809.0,C,145.5,4,HBACK,0.891,0,0.0
60388,1/2/2020,M,4.0,681.0,D,86.3,1,HBACK,1.881,0,0.0
60389,1/2/2020,M,4.0,773.0,F,110.0,1,STNWG,5.170,0,0.0
60390,1/2/2020,M,3.0,714.0,B,NaN,1,HBACK,1.903,0,0.0


## 2. Fill empty cells with Median
Since the numclaims column contain '0's which will be removed if we were to use drop.na(), we will fill up the empty cells with Median value instead to prevent excessive loss of data.

In [7]:
# fill empty values in Multiple Columns with Median
data[['agecat', 'credit_score', 'traffic_index']] = data[['agecat', 'credit_score', 'traffic_index']].fillna(data[['agecat', 'credit_score', 'traffic_index']].median())

In [8]:
data

,pol_eff_dt,gender,agecat,credit_score,area,traffic_index,veh_age,veh_body,veh_value,numclaims,claimcst0
0,12/30/2020,F,4.0,381.0,D,133.6,2,HBACK,1.331,0,0.0
1,12/30/2020,F,2.0,549.0,D,163.6,1,UTE,3.740,0,0.0
2,12/30/2020,M,2.0,649.0,B,117.5,4,COUPE,0.880,0,0.0
3,12/30/2020,M,4.0,743.0,B,100.7,3,SEDAN,1.045,0,0.0
4,12/30/2020,F,4.0,817.0,C,115.5,4,HBACK,0.473,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
60387,1/2/2020,F,5.0,809.0,C,145.5,4,HBACK,0.891,0,0.0
60388,1/2/2020,M,4.0,681.0,D,86.3,1,HBACK,1.881,0,0.0
60389,1/2/2020,M,4.0,773.0,F,110.0,1,STNWG,5.170,0,0.0
60390,1/2/2020,M,3.0,714.0,B,108.3,1,HBACK,1.903,0,0.0


## 3. Splitting Dataset into Two
For the purpose of our analysis, it is best suited if we split our dataset into two depending on the Machine Learning Techniques that the variables are involved in:

DataFrame containing variables used in Linear Regression (reg_df)
DataFrame containing variables used in Clustering (clust_df)

From this point, the further data cleaning and preparation is done separately for these two DataFrames

Variables used in Linear Regression:

gender, agecat, traffic_index (3 in total)

Variables used in Clustering:

gender, agecat, credit_score, area, traffic_index, veh_age, veh_body, veh_value, numclaims, claimcst0 (10 in total)


In [9]:
reg_df = data[[
    'gender', 
    'agecat',
    'traffic_index',
]]

clust_df = data[[
    'gender',
    'agecat',
    'credit_score',
    'area',
    'traffic_index',
    'veh_age',
    'veh_body',
    'veh_value',
    'numclaims',
    'claimcst0'
]]

In [10]:
reg_df.info

<bound method DataFrame.info of       gender  agecat  traffic_index
0          F     4.0          133.6
1          F     2.0          163.6
2          M     2.0          117.5
3          M     4.0          100.7
4          F     4.0          115.5
...      ...     ...            ...
60387      F     5.0          145.5
60388      M     4.0           86.3
60389      M     4.0          110.0
60390      M     3.0          108.3
60391      F     3.0          114.9

[60392 rows x 3 columns]>

In [11]:
clust_df.info

<bound method DataFrame.info of       gender  agecat  credit_score area  traffic_index  veh_age veh_body  \
0          F     4.0         381.0    D          133.6        2    HBACK   
1          F     2.0         549.0    D          163.6        1      UTE   
2          M     2.0         649.0    B          117.5        4    COUPE   
3          M     4.0         743.0    B          100.7        3    SEDAN   
4          F     4.0         817.0    C          115.5        4    HBACK   
...      ...     ...           ...  ...            ...      ...      ...   
60387      F     5.0         809.0    C          145.5        4    HBACK   
60388      M     4.0         681.0    D           86.3        1    HBACK   
60389      M     4.0         773.0    F          110.0        1    STNWG   
60390      M     3.0         714.0    B          108.3        1    HBACK   
60391      F     3.0         682.0    D          114.9        4    STNWG   

       veh_value  numclaims  claimcst0  
0          1.3

## 4. Removing Outliers
Outliers increase the variability in your data, which decreases statistical power. Consequently, excluding outliers can cause your results to become statistically significant.

In [12]:
def removeOutliers(df, var):
    q1 = df[var].quantile(0.25)
    q3 = df[var].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    df = df[df[var] < upper]
    df = df[df[var] > lower]
    return df

In [13]:
data = removeOutliers(data, "credit_score")
data = removeOutliers(data, "traffic_index")
data = removeOutliers(data, "veh_value")
data

,pol_eff_dt,gender,agecat,credit_score,area,traffic_index,veh_age,veh_body,veh_value,numclaims,claimcst0
0,12/30/2020,F,4.0,381.0,D,133.6,2,HBACK,1.331,0,0.0
1,12/30/2020,F,2.0,549.0,D,163.6,1,UTE,3.740,0,0.0
2,12/30/2020,M,2.0,649.0,B,117.5,4,COUPE,0.880,0,0.0
3,12/30/2020,M,4.0,743.0,B,100.7,3,SEDAN,1.045,0,0.0
4,12/30/2020,F,4.0,817.0,C,115.5,4,HBACK,0.473,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
60386,1/2/2020,F,3.0,755.0,E,34.5,4,UTE,2.310,0,0.0
60387,1/2/2020,F,5.0,809.0,C,145.5,4,HBACK,0.891,0,0.0
60388,1/2/2020,M,4.0,681.0,D,86.3,1,HBACK,1.881,0,0.0
60390,1/2/2020,M,3.0,714.0,B,108.3,1,HBACK,1.903,0,0.0


## 5. Addition of 4 New Derived Columns: "month", "year", "monthDiff", "claims"
1. month and year: for calculation of monthDiff
2. monthDiff: difference in months from pol_eff_date to current (1 april 2023)
3. claims: indicated by "Yes" and "No", to specify if a customer have claims or not

In [14]:
# getting columns "month" and "year"
for i , row in data.iterrows():
    data._set_value(i, "month", int(row["pol_eff_dt"][:row["pol_eff_dt"].find("/")]))
    data._set_value(i, "year", int(row["pol_eff_dt"][(row["pol_eff_dt"].rfind("/") + 1):]))

In [15]:
# getting column "monthDiff"
for i , row in data.iterrows():
    if row["month"] > 4:
        x = 16
    else:
        x = 4
    data._set_value(i, "monthDiff", (x - row["month"]) + (2023 - row["year"]) * 12)
    
data.describe()

,agecat,credit_score,traffic_index,veh_age,veh_value,numclaims,claimcst0,month,year,monthDiff
count,54135.000000,54135.000000,54135.000000,54135.000000,54135.000000,54135.000000,54135.000000,54135.000000,54135.0,54135.000000
mean,3.476512,677.642930,104.557933,2.751843,1.717331,0.194107,432.579089,6.527976,2020.0,41.538302
std,1.366029,116.232054,32.422771,1.044581,0.866484,0.526960,2423.408665,3.443605,0.0,3.448855
min,1.000000,365.000000,10.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2020.0,36.000000
25%,3.000000,617.000000,82.500000,2.000000,1.078000,0.000000,0.000000,4.000000,2020.0,39.000000
50%,3.000000,682.000000,108.300000,3.000000,1.584000,0.000000,0.000000,7.000000,2020.0,42.000000
75%,4.000000,771.000000,130.500000,4.000000,2.167000,0.000000,0.000000,10.000000,2020.0,45.000000
max,6.000000,850.000000,198.800000,4.000000,4.272400,5.000000,69478.897830,12.000000,2020.0,47.000000


In [16]:
# getting column "claims"
for i, row in data.iterrows():
    if row["numclaims"] == 0:
        data._set_value(i, "claims", "No")
    else:
        data._set_value(i, "claims", "Yes")

## 6. Scaling credit_score, traffic_index & veh_age to a range 1-100
Linear transformation of data that maps the minimum value to maximum value to 1 and 100 respectively.

In [17]:
# import necessary library
from sklearn.preprocessing import MinMaxScaler

In [18]:
# create scaler
scaler = MinMaxScaler(feature_range=(1,100))
# scale feature
data[['credit_score']] = scaler.fit_transform(data[['credit_score']])
# show feature
data[['credit_score']]

,credit_score
0,4.265979
1,38.558763
2,58.971134
3,78.158763
4,93.263918
...,...
60386,80.608247
60387,91.630928
60388,65.503093
60390,72.239175


In [19]:
# create scaler
scaler = MinMaxScaler(feature_range=(1,100))
# scale feature
data[['traffic_index']] = scaler.fit_transform(data[['traffic_index']])
# show feature
data[['traffic_index']]

,traffic_index
0,65.811441
1,81.542373
2,57.369174
3,48.559852
4,56.320445
...,...
60386,13.846928
60387,72.051377
60388,41.009004
60390,52.545021


In [20]:
# create scaler
scaler = MinMaxScaler(feature_range=(1,100))
# scale feature
data[['veh_age']] = scaler.fit_transform(data[['veh_age']])
# show feature
data[['veh_age']]

,veh_age
0,34.0
1,1.0
2,100.0
3,67.0
4,100.0
...,...
60386,100.0
60387,100.0
60388,1.0
60390,1.0


In [21]:
# Exporting DF to CSV file named "Vehicle_policies_2020.csv"
data.to_csv("/Users/nataliecje/Desktop/SC1015 Lab/Mini Project/Prepped.csv", index = False)